In [2]:
import numpy as np
from matplotlib import pyplot as plt

import petab.v1.C as C
import pypesto
import pypesto.optimize as optimize
import pypesto.petab
import pypesto.sample as sample
import pypesto.visualize as visualize

# from polypesto._petab import CRP2_CPE as crp
from polypesto.core.params import ParameterContainer
from polypesto.core.pypesto import create_problem_set, load_pypesto_problem
from polypesto.utils.plot import plot_all_measurements

from polypesto.models.CRP2 import irreversible_cpe
# from experiments.irreversible_cpe.problem import ds_0

%load_ext autoreload
%autoreload 2

In [ ]:
pc = ParameterContainer.from_json("/PolyPESTO/experiments/irreversible_cpe/params.json")
pg = pc.get_parameter_group("IRREVERSIBLE")

# paths = create_problem_set(irreversible_cpe, pg, ds_0(), force_compile=False)

Visualization table not available. Skipping.
Visualization table not available. Skipping.
Visualization table not available. Skipping.
Visualization table not available. Skipping.
Compiling amici model to folder /PolyPESTO/amici_models/0.30.0/irreversible_cpe.
2025-02-20 09:41:42.319 - amici.petab.sbml_import - INFO - Importing model ...
2025-02-20 09:41:42.320 - amici.petab.sbml_import - INFO - Validating PEtab problem ...
Visualization table not available. Skipping.
2025-02-20 09:41:42.332 - amici.petab.sbml_import - INFO - Model name is 'irreversible_cpe'.
Writing model code to '/PolyPESTO/amici_models/0.30.0/irreversible_cpe'.
2025-02-20 09:41:42.333 - amici.petab.sbml_import - INFO - Species: 1
2025-02-20 09:41:42.333 - amici.petab.sbml_import - INFO - Global parameters: 14
2025-02-20 09:41:42.334 - amici.petab.sbml_import - INFO - Reactions: 0
2025-02-20 09:41:42.358 - amici.petab.sbml_import - INFO - Observables: 2
2025-02-20 09:41:42.359 - amici.petab.sbml_import - INFO - Sigma

Creating SBML model: irreversible_cpe
Creating irreversible CRP parameters.
                 filename : /PolyPESTO/experiments/irreversible_cpe/data/ds_0/petab/common/irreversible_cpe.xml
         file size (byte) : 5840
           read time (ms) : 1.607656
        c-check time (ms) : 0.631094
      validation error(s) : 0
    (consistency error(s)): 0
    validation warning(s) : 0
  (consistency warning(s)): 0
PETAB IMPORTER: Output folder:  /PolyPESTO/amici_models/0.30.0/irreversible_cpe
PRINTING OUTPUT FOLDER /PolyPESTO/amici_models/0.30.0/irreversible_cpe


2025-02-20 09:41:42.388 - amici.sbml_import - DEBUG - Finished processing SBML annotations        ++ (1.83E-04s)
2025-02-20 09:41:42.395 - amici.sbml_import - DEBUG - Finished gathering local SBML symbols       ++ (8.21E-04s)
2025-02-20 09:41:42.407 - amici.sbml_import - DEBUG - Finished processing SBML parameters         ++ (6.58E-03s)
2025-02-20 09:41:42.413 - amici.sbml_import - DEBUG - Finished processing SBML compartments       ++ (1.76E-04s)
2025-02-20 09:41:42.422 - amici.sbml_import - DEBUG - Finished processing SBML species initials  +++ (2.39E-03s)
2025-02-20 09:41:42.428 - amici.sbml_import - DEBUG - Finished processing SBML rate rules        +++ (1.44E-03s)
2025-02-20 09:41:42.429 - amici.sbml_import - DEBUG - Finished processing SBML species            ++ (1.18E-02s)
2025-02-20 09:41:42.433 - amici.sbml_import - DEBUG - Finished processing SBML reactions          ++ (7.85E-04s)
2025-02-20 09:41:42.441 - amici.sbml_import - DEBUG - Finished processing SBML rules            

running build_ext
------------------------------ model_ext ------------------------------
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test CUR_FLAG_SUPPORTED
-- Performing Test CUR_FLAG_SUPPORTED - Success
-- Performing Test CUR_FLAG_SUPPORTED
-- Performing Test CUR_FLAG_SUPPORTED - Success
-- Performing Test CUR_FLAG_SUPPORTED
-- Performing Test CUR_FLAG_SUPPORTED - Success
-- Performing Test CUR_FLAG_SUPPORTED
-- Performing Test CUR_FLAG_SUPPORTED - Success
-- Appending flags from AMICI_CXXFLAGS: -fopenmp
-- Appending

/usr/local/lib/python3.10/dist-packages/amici/petab/simulations.py:209: RuntimeWarning: The following problem parameters were not used: {'rX'}
  fill_in_parameters(
/usr/local/lib/python3.10/dist-packages/amici/petab/simulations.py:209: RuntimeWarning: The following problem parameters were not used: {'rX'}
  fill_in_parameters(


SIMULATING PETAB MODEL!!!!!


/usr/local/lib/python3.10/dist-packages/amici/petab/simulations.py:209: RuntimeWarning: The following problem parameters were not used: {'rX'}
  fill_in_parameters(


In [ ]:
p_id = "p5"
# yaml_path = paths[p_id]
yaml_path = "/PolyPESTO/src/data/fitting/CRP2_CPE/petab/p5/petab.yaml"
importer, problem = load_pypesto_problem(yaml_path, "CRP2_CPE", force_compile=False)

In [ ]:
plot_all_measurements(
    importer.petab_problem.measurement_df,
    # group_by=C.SIMULATION_CONDITION_ID,
    group_by=C.OBSERVABLE_ID,
    format_axes_kwargs={
        "set_xlabel": "Total Conversion",
        "set_ylabel": "Monomer Conversion",
        # "set_xlim": (0, 1),
        # "set_ylim": (0, 1),
    },
    plot_style="both",
    alpha=0.5,
)
plt.show()

In [ ]:
# optimizer = optimize.ScipyOptimizer(method="COBYLA")  # method="Nelder-Mead")
optimizer = optimize.PyswarmOptimizer()
history_options = pypesto.HistoryOptions(
    trace_record=True, storage_file="history_{id}.csv"
)
engine = pypesto.engine.MultiProcessEngine()
n_starts = 10

# run optimization of problem 1
import amici

problem.startpoint_method = pypesto.startpoint.LatinHypercubeStartpoints()
problem.objective.amici_solver.setSensitivityMethod(amici.SensitivityMethod.none)
problem.objective.amici_solver.setSensitivityOrder(amici.SensitivityOrder.none)
problem.objective.amici_solver.setMaxStepSize(1e-3)
problem.objective.amici_model.setAlwaysCheckFinite(True)
# problem.objective.amici_solver.setSensitivityOrder(1)
# problem.objective.amici_solver.
result = optimize.minimize(
    problem=problem,
    optimizer=optimizer,
    n_starts=n_starts,
    engine=engine,
    history_options=history_options,
)

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result.summary()))

In [ ]:
problem.objective.amici

In [ ]:
sampler = sample.AdaptiveParallelTemperingSampler(
    internal_sampler=sample.AdaptiveMetropolisSampler(),
    n_chains=3,
)

n_samples = 10000
result = sample.sample(
    problem,
    n_samples=n_samples,
    sampler=sampler,
    x0=np.array([1, 1]),
    filename=None,
)
elapsed_time = result.sample_result.time
print(f"Elapsed time: {round(elapsed_time,2)}")

In [ ]:
ax = visualize.sampling_parameter_traces(result, use_problem_bounds=True, size=(12, 5))

p = pg.by_id("p5").to_dict()
# ax[0][0].plot([0, n_samples], np.log10([p["rA"], p["rA"]]), "r--")
# ax[0][1].plot([0, n_samples], np.log10([p["rB"], p["rB"]]), "r--")
ax[0][0].plot([0, n_samples], ([p["rA"], p["rA"]]), "r--")
ax[0][1].plot([0, n_samples], ([p["rB"], p["rB"]]), "r--")
# ax[1][0].plot([0, n_samples], [p["KAA"], p["KAA"]], "r--")
# ax[1][1].plot([0, n_samples], [p["KBA"], p["KBA"]], "r--")